# Instalación

**Los datos necesarios para realizar la práctica están en una carpeta compartida que se llama `data`, situada en "Shared with me". Para que el cuaderno pueda acceder a ella es necesario crear un link en "My Drive". El link se crea simplemente seleccionando "data", y desde el menu del botón derecho seleccione "Add a shortcut to Drive".**

**Una vez hecho esto, ejecute la siguiente celda. Esta celda montará el directorio de trabajo del cuaderno y verá la carpeta `data` a través del link. Para montar el directorio le pedirá que se registre en "Drive"**.


In [ ]:
## Instalación: esta celda monta el directorio de trabajo
if 'google.colab' in str(get_ipython()):
    # Setup colab
    from google.colab import drive
    drive.mount('/drive')
    %cd /drive/My Drive/data
else:
    pass

# Introducción

La mayoría de los avances en las áreas de la inteligencia artificial y visión por computador se han producido de la mano del *deep learning*. En algunas aplicaciones, los sistemas basados en *deep learning* se aproximan a las capacidades del ser humano, y en muchos casos las superan con creces. Algunos ejemplos son: traducción del lenguaje, reconocimiento y detección de objetos en imágenes, videojuegos... El *deep learning* es una rama de Machine Learning (ML) que se caracteriza por el uso de redes neuronales de gran tamaño y con multitud de capas, las cuales requieren una gran capacidad de cálculo. Estas estructuras permiten desentrañar los patrones complejos y jerárquicos que gobiernan las relaciones entre los datos que nos rodean. Las redes neuronales están compuestas de neuronas, que son unidades de cálculo muy simples. Sin embargo, la complejidad de las conexiones crean estructuras de cálculo altamente flexibles y fácilmente "escalables", lo que las convierte en estructuras computacionalmente muy potentes. Estas estructuras se adaptan a una aplicación concreta mediante el entrenamiento con una base de datos. A menudo los datos surgen de la recopilación organizada en problemas de la vida real. 

En este módulo aprenderemos a construir redes neuronales mediante **TensorFlow**, que es es una biblioteca de código abierto desarrollado por Google para construir y entrenar redes neuronales. Además, también utilizaremos **Keras**, que es una herramienta de más alto nivel, que se ejecuta sobre **Tensorflow** y cuyo objetivo es la simplificación del código, de modo que se pueda experimentar rápidamente con el máximo de funcionalidad. Estas herramientas están diseñadas para realizar cálculos en la GPU de forma muy optimizada. 

Los objetivos de este módulo son:
- crear una arquitectura de red totalmente conectada: **fully-connected**
- aplicar las redes neuronales a dos problemas clásicos del Machine Learning: **regresión** y **clasificación**
- entrenar las redes mediante el algoritmo **stochastic gradient descent**
- mejorar la eficiencia del entrenamiento mediante diferentes técnicas de *regularización*, entre las que se encuentran el **dropout** y **batch normalization**.

Todo ello lo haremos con diferentes ejemplos de casos reales. En cada sesión se propone la lectura del tutorial asociado, para pasar a resolver el cuaderno práctico, respondiendo a las preguntas planteadas.

# Análisis de una neurona
En esta sección se analiza la unidad fundamental de la red, la neurona. Un diagrama de una única neurona con una única entrada es:

<figure style="padding: 1em;"><center>
<img src="https://i.imgur.com/mfOlDR6.png" width="250" alt=""></center>
<figcaption style="textalign: center; font-style: italic"><center>La unidad lineal: y = w x + b
</center></figcaption>
</figure>

La entrada es `x` y su conexión con la neurona se realiza mediante un peso (**weight**) `w`. Siempre que se introduzca un dato de entrada se multiplica por el peso de la conexión, llegando a la neurona el valor $w x + b$. La neurona debe aprender a configurar el peso de la conexión para realizar una tarea predefinida. `b` es un peso especial que se denomina **offset**, **sesgo** o **bias**. El **bias** no tiene asociado ningún dato de entrada, por eso se representa un `1` en el diagrama, de modo que a la neurona le llega simplemente un offset `b`. El **bias** es otro parámetro que la neurona deberá aprender, y es importante porque permite a la neurona modificar la salida independientemente del valor de las entradas. El valor `y` es el valor de salida de la neurona, que se obtiene mediante la suma de todas las posibles entradas o conexiones más el valor del **bias**. En el caso de una única entrada o conexión, la salida se obtiene mediante la ecuación $y = w x + b$.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">

<strong>¿Te resulta familiar la fórmula $y=w x + b$?</strong><br>
Es la ecuación de una recta, donde $w$ es la pendiente y $b$ es el punto de corte en el origen. 
</blockquote>

## Ejemplo de funcionamiento

Una única neurona no tiene interés práctico, ya que solo puede modelar relaciones lineales, por lo que también se llama unidad lineal. Aún así, su análisis es interesante para establecer las bases del funcionamiento. Vamos a analizar el comportamiento de una neurona con una base de datos que contiene las características de 80 barritas de cereales donde se quiere examinar la relación entre el contenido en azúcar de cada barrita con sus calorías, la base de datos se llama "80 Cereals". La base de datos es una matriz, donde las filas corresponden a 80 muestras de barritas que han sido analizadas. Las columnas corresponden a las diferentes características de cada una de las barritas analizadas, por ejemplo en una columna se almacenan los gramos de azúcar por ración y en otra las calorías por ración. Podríamos usar esos datos para entrenar la neurona y obtener la relación lineal que mejor se ajusta a los datos, por ejemplo, podríamos obtener un valor para el bias de `b=90` y un peso de `w=2.5`. Una vez entrenada la neurona con esos datos ya tenemos el modelo, y con ese modelo podríamos estimar las calorías para cualquier tipo de barrita, por ejemplo, una barrita con 5 gramos de azúcar por ración tendría: 

$$\text{calories} = w\cdot x + b = 2.5 \cdot 5 + 90 = 102.5.$$ 
En la figura se puede ver la estructura del cálculo:

<figure style="padding: 1em;"><center>
<img src="https://i.imgur.com/yjsfFvY.png" width="1000" alt="">
<figcaption style="textalign: center; font-style: italic"><center>Estructura de cálculo de una neurona entrenada.
</center></figcaption></center>
</figure>


## Múltiples entradas

Además del contenido en azúcar, la base de datos "80 Cereals" tiene muchas más características. Si queremos hacer nuestro análisis más preciso tendríamos que considerar otros aspectos de las barritas, como pueden ser la cantidad de fibra o de proteínas. Algunas de ellas están a nuestra disposición en la base de datos. Esta generalización de nuestro modelo es muy sencilla, basta con añadir diferentes conexiones de entrada en la neurona, una por cada característica a considerar. Para obtener la salida solo tenemos que multiplicar cada entrada por el peso de su conexión y sumar todas ellas, junto con el bias:

<figure style="padding: 1em;">
<center><img src="https://i.imgur.com/vyXSnlZ.png" width="300" alt=" "></center>
<figcaption style="textalign: center; font-style: italic"><center>Una neurona con tres entradas, x0, x1, and x2, junto con el 'bias'.
</center></figcaption>
</figure>

El esquema de cálculo de esta neurona sería: 
$$y = w_0 x_0 + w_1 x_1 + w_2 x_2 + b.$$

Como se puede observar, una neurona con una entrada genera la ecuación de una recta, si ponemos dos entradas genera un plano y si ponemos más entradas tenemos un hiperplano en un espacio multidimensional.


## Configurar una neurona en Keras

El modo más sencillo de crear un modelo computacional en Keras es mediante la instrucción `keras.Sequential`, el cual crea una red neuronal como una pila de capas que actúan de forma secuencial, cada una con el número de neuronas que queramos. Podemos crear modelos como el anterior mediante una capa donde todas las entradas están conectadas a todas las neuronas, esto se denomina capa densa, `dense`. Existen muchos tipos de capas, algunos de ellos los veremos posteriormente. A continuación vamos a definir un modelo lineal de una sola neurona con tres características de entrada (`'sugars'`, `'fiber'`, and `'protein'`) y una salida (`'calories'`):

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Create a network with 1 linear unit
model = keras.Sequential([layers.Dense(units=1, input_shape=[3])])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1)                 4         
                                                                 
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


El primer argumento, `units`, define el número de neuronas, que es el número de salidas de la capa (una neurona solo tiene una salida). En este caso queremos modelar las  `'calories'`, por tanto usamos `units=1`. El segundo argumento, `input_shape`, configura el número de entradas, o la dimensión del modelo. En nuestro caso `input_shape=[3]`, ya que el modelo considera únicamente tres entradas o características: `'sugars'`, `'fiber'`, and `'protein'`. Ya tenemos el modelo configurado para  entrenar la neurona.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
<strong>Origen de los datos:</strong>
    
Los datos que usaremos provienen típicamente de bases de datos o tablas, como por ejemplo un `dataframe` de `Pandas`. Las filas suelen corresponder a las muestras, observaciones o experimentos realizados. Las muestras tienen propiedades o características, que son entradas en forma de columnas en la base de datos. Si consideramos todas las características disponibles, tendríamos que poner el número total de columnas, exceptuando la característica que queremos modelar, que adoptaría el papel de salida: ```input_shape=[num_columns-1]```. Para configurar el valor de ```input_shape```, Keras usa una `lista` de Python. Las listas son arrays y permiten configurar estructuras de datos multidimensionales, por ejemplo podríamos tener como entradas las imágenes que genera una cámara, las cuales tienen tres dimensiones:
```[height, width, channels]```.
</blockquote>

# Redes neuronales
Como se ha visto anteriormente, una neurona puede simular una relación lineal entre las entradas y la salida. Ahora se trata de combinar neuronas para modelar relaciones más complejas, dando lugar a una red neuronal. Las redes neuronales son capaces de extraer las relaciones que pueden existir entre los datos de entrada, aunque no sean necesariamente lineales. Esto se debe a que la redes neuronales incluyen operaciones no lineales que veremos en breve. 

## Capas
Las redes están organizadas por **capas** de neuronas o unidades lineales. Como hemos mencionado, cuando las neuronas de una capa comparten todas las entradas, es decir todas las neuronas de una capa están conectadas con todas las neuronas de la otra se dice que la capa es **densa**.

<figure style="padding: 1em;"><center>
<img src="https://i.imgur.com/2MA4iMV.png" width="300" ></center>
<figcaption style="textalign: center; font-style: italic"><center>Una capa de entrada conectada con una capa densa de dos neuronas.
</center></figcaption>
</figure>

Cada capa de la red neuronal realiza transformaciones relativamente sencillas. Sin embargo, a través de una pila profunda de capas, la red puede transformar sus entradas de un modo cada vez más complejo. En una red bien entrenada, el resultado de cada capa es una aproximación sucesiva a la solución.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
    <strong>Existen muchos tipos de capas</strong><br>
    
Una "capa" en Keras es una estructura muy general, de hecho puede ser cualquier tipo de transformación de datos, incluso transformaciones que no están definidas por neuronas. Algunos ejemplos de capas típicas construidas mediante neuronas son las <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D">convolucionales</a> y las <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN">recurrentes</a>, las cuales se diferencian en la estructura de sus conexiones. Otras capas se usan para <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding">extraer características (feature engeenering)</a> o para realizar <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add">operaciones aritméticas</a>. Existe una gran variedad de capas <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers">en Keras</a>
</blockquote>


## Función de activación
Como hemos visto, una capa densa permite establecer un modelo lineal, además, se puede demostrar que la concatenación de capas densas es una suma de modelos lineales, los cuales dan lugar a otro modelo lineal. Es decir, solo podremos extraer relaciones lineales entre las variables de entrada, lo cual no es muy flexible. Para modelar relaciones complejas es necesario dotar a las redes de elementos que permitan romper la linealidad de la estructura. Uno de estos elementos son las funciones de activación.

<figure style="padding: 1em;">
<center><img src="https://i.imgur.com/OLSUEYT.png" width="400" alt=" ">
<figcaption style="textalign: center; font-style: italic"><center>Para poder aprender a modelar curvas necesitamos funciones de activación. 
</center></figcaption>
</figure>

Una **función de activación** es simplemente una función no lineal que aplicamos a cada una de las salidas de una capa (normalmente a cada neurona o unidad lineal). La función de activación más común es la *rectificadora*, definida como: $f(x) = \max(0, x)$.

<figure style="padding: 1em;">
<center><img src="https://i.imgur.com/aeIyAlF.png" width="400" alt=" ">
<figcaption style="textalign: center; font-style: italic"><center>
Función rectificadora, la salida es igual a la entrada (y=x) cuando x>0 y es nula (y=0) cuando x<0.
</center></figcaption>
</figure>

La función rectificadora devuelve el valor de la entrada, si esta es positiva, y 0 si la entrada es negativa. Es una función que "rectifica" la parte negativa a cero. La rectificación es la responsable de la pérdida de linealidad. Cuando esta función es aplicada a la salida de una neurona se llama **unidad lineal rectificada** o **ReLU**. Al aplicar una activación ReLU a una unidad lineal hace que la salida se transforme en $f(x) = \max(0, w \cdot x + b)$, lo cual se describe gráficamente como:

<figure style="padding: 1em;">
<center><img src="https://i.imgur.com/eFry7Yu.png" width="250" alt=" ">
<figcaption style="textalign: center; font-style: italic"><center>Diagrama de una unidad lineal rectificada, en vez de representarse con un símbolo '+' se suele representar con un símbolo '_/'.
</center></figcaption>
</figure>

## Pilas de capas densas
Ahora que tenemos capas de unidades no lineales, vamos a establecer una estructura de pilas de capas para obtener transformaciones más complejas.

<figure style="padding: 1em;">
<center><img src="https://i.imgur.com/Y5iwFQZ.png" width="450" alt="">
<figcaption style="textalign: center; font-style: italic"><center>Una capa de entrada, junto con dos capas ocultas y una capa con un elemento lineal de salida. Una pila de capas densas da lugar a una red completamente conectada, la cual se denomina "fully-connected network".
</center></figcaption>
</figure>

Las capas entre la entrada y salida se suelen llamar capas **ocultas**, ya que en principio solo vemos los datos de entrada y salida. Nótese que la capa final solo contiene una unidad lineal, sin función de activación. Esto hace que la red sea adecuada para taras como la regresión, donde la salida debe tomar un valor numérico arbitrario (negativos incluidos). 

## Construcción de un modelo secuencial

El modelo `Secuencial` descrito anteriormente conecta una pila de capas en orden, desde la primera a la última: la primera admite los datos de entrada y la última capa genera la salida. El modelo de la figura anterior se programa del siguiente modo:

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    # the hidden ReLU layers
    layers.Dense(units=4, activation='relu', input_shape=[2]),
    layers.Dense(units=3, activation='relu'),
    # the linear output layer 
    layers.Dense(units=1),
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 4)                 12        
                                                                 
 dense_2 (Dense)             (None, 3)                 15        
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


Obsérvese que las capas se pasan en forma de lista `[capa1, capa2, capa3, ...]`. Para añadir una función de activación a una capa, basta con dar su nombre en el argumento `activation`.